In [13]:
#インポート
from bs4 import BeautifulSoup as bs
import re
import time
import os
import sys
import json
import pandas as pd
import numpy as np
import pickle
import unicodedata
import itertools
from openai import AzureOpenAI
from dotenv import load_dotenv
load_dotenv()
api_version="2023-12-01-preview"
engine = 'gpt-4o-mini'
gpt = AzureOpenAI(
    api_key=os.getenv("OPENAI_API_KEY_GPT4OMINI"),  
    api_version=api_version,
    azure_endpoint = os.getenv("OPENAI_API_BASE_GPT4OMINI")
    )

import os
from openai import OpenAI
plamo = OpenAI(
    base_url="https://platform.preferredai.jp/api/completion/v1",
    api_key='NjcyYWFhNjUyZWZlMzAwZjJmMzEyN2FlOkRyeDB3enhyQWJsSm9rcENMdFR0WDhYanRrMDc2RnEy'
    # other params...,
)


data = pd.read_csv('../data.csv')
import json
import enum
from typing_extensions import TypedDict
import typing_extensions as typing
from typing import List
from pydantic import BaseModel, Field

In [4]:
class make_content():
    def __init__(self, path):
        # 各doc_idのパスを指定
        self.path = path
        self.doc_id = path.split('/')[-1]
    
    def edit_text(self, text):
        return re.sub(r'[^\u3040-\u309F\u30A0-\u30FF\uFF65-\uFF9F\u4E00-\u9FFF]', '', text)
    
    def make_content(self):
        try:
            with open(f"../data/contents/{self.doc_id}.pkl", 'rb') as f:
                dct = pickle.load(f)
                f.close()
            return dct
        except:
            dct = pd.DataFrame()
            files = os.listdir(self.path)
            tmp_h1 = ''
            tmp_h2 = ''
            tmp_h3 = ''
            tmp_h4 = ''
            tmp_h5 = ''
            tmp_h6 = ''
            if not os.path.exists(f'../data/{self.doc_id}'):
                os.makedirs(f'../data/{self.doc_id}')
            for file in files:
                with open(f'{self.path}/{file}', 'r') as f:
                    soup = bs(f, 'html.parser')
                    f.close()
                html_tag = soup.find_all(['h1', 'h2', 'h3', 'h4', 'h5', 'h6', 'table'])
                for index, tag in enumerate(html_tag):
                    if tag.name == 'h1':
                        text = unicodedata.normalize('NFKC', tag.get_text()).replace('\n', '').replace(' ', '')
                        if text == '':
                            continue
                        tmp_h1 = self.edit_text(text)
                        tmp_h2 = ''
                        tmp_h3 = ''
                        tmp_h4 = ''
                        tmp_h5 = ''
                        tmp_h6 = ''
                    elif tag.name == 'h2':
                        text = unicodedata.normalize('NFKC', tag.get_text()).replace('\n', '').replace(' ', '')
                        if text == '':
                            continue
                        tmp_h2 = self.edit_text(text)
                        tmp_h3 = ''
                        tmp_h4 = ''
                        tmp_h5 = ''
                        tmp_h6 = ''
                    elif tag.name == 'h3':
                        text = unicodedata.normalize('NFKC', tag.get_text()).replace('\n', '').replace(' ', '')
                        if text == '':
                            continue
                        tmp_h3 = self.edit_text(text)
                        tmp_h4 = ''
                        tmp_h5 = ''
                        tmp_h6 = ''
                    elif tag.name == 'h4':
                        text = unicodedata.normalize('NFKC', tag.get_text()).replace('\n', '').replace(' ', '')
                        if text == '':
                            continue
                        tmp_h4 = self.edit_text(text)
                        tmp_h5 = ''
                        tmp_h6 = ''
                    elif tag.name == 'h5':
                        text = unicodedata.normalize('NFKC', tag.get_text()).replace('\n', '').replace(' ', '')
                        if text == '':
                            continue
                        tmp_h5 = self.edit_text(text)
                        tmp_h6 = ''
                    elif tag.name == 'h6':
                        text = unicodedata.normalize('NFKC', tag.get_text()).replace('\n', '').replace(' ', '')
                        if text == '':
                            continue
                        tmp_h6 = self.edit_text(text)
                    elif tag.name == 'table':
                        dct =pd.concat([dct, pd.DataFrame({'h1': [tmp_h1], 'h2': [tmp_h2], 'h3': [tmp_h3], 'h4': [tmp_h4], 'h5': [tmp_h5], 'h6': [tmp_h6], 'table_id':[tag.get('table-id')],'html': [file]})])
            pickle.dump(dct, open(f"../data/contents/{self.doc_id}.pkl", 'wb'))
            return dct

In [6]:
"""for _, rows in data.iterrows():
    path = f'../{rows["flag"]}/reports/{rows["doc_id"]}'
    make_content(path).make_content()"""

In [61]:
class search_table():
    def __init__(self, words, doc_id, df_train, question):
        self.words = words
        self.doc_id = doc_id
        self.df_train = df_train
        self.question = question

    # 類似質問から目次データを取得
    def get_contents(self, ):
        #重複単語数でランク付けしたほうがいいかもしれない
        f = lambda x:sum([(word in x) for word in self.words]) 
        similary_tables = self.df_train[self.df_train['words'].apply(f) == self.df_train['words'].apply(f).max()]['table_id'].values
        
        df_contents = pd.DataFrame()
        for table_id in similary_tables:
            doc_id = table_id.split('-')[0]
            with open(f"../data/contents/{doc_id}.pkl", 'rb') as f:
                dct = pickle.load(f)
                f.close()
            df_contents = pd.concat([df_contents, dct.query('table_id == @table_id')], axis=0)

        return df_contents.drop_duplicates(subset=['h1', 'h2', 'h3', 'h4', 'h5'])
    
    #類似質問の目次データを該当文書のtable_id候補を取得する
    def get_tables(self,):
        contents = self.get_contents()
        contents.reset_index(inplace=True)
        with open(f"../data/contents/{self.doc_id}.pkl", 'rb') as f:
            myself_contents = pickle.load(f)
            f.close()
            
        if contents.empty:
            return myself_contents['table_id'].values
    
        tables_dict= {}
        for index, rows in contents.iterrows():
            tmp = myself_contents
            tmp_header = 'h0'
            if rows['h1'] != '' and not myself_contents.query('h1 == @rows["h1"]').empty:
                tmp = myself_contents.query('h1 == @rows["h1"]',)
                tmp_header = 'h1'
                
            if rows['h2'] != '' and not myself_contents.query('h2 == @rows["h2"]').empty:
                tmp = myself_contents.query('h2 == @rows["h2"]',)
                tmp_header = 'h2'

            if rows['h3'] != '' and not myself_contents.query('h3 == @rows["h3"] or h4 == @rows["h3"]').empty:
                tmp = myself_contents.query('h3 == @rows["h3"] or h4 == @rows["h3"]')
                tmp_header = 'h3'
            
            if rows['h4'] != '' and not myself_contents.query('h4 == @rows["h4"] or h5 == @rows["h4"]').empty:
                tmp = myself_contents.query('h4 == @rows["h4"] or h5 == @rows["h4"]')
                tmp_header = 'h4'

            if rows['h5'] != '' and not myself_contents.query('h5 == @rows["h5"] or h4 == @rows["h5"]').empty:
                tmp = myself_contents.query('h5 == @rows["h5"] or h4 == @rows["h5"]',)
                tmp_header = 'h5'
            
            tables_dict[f'no{index}'] = {'depth':tmp_header, 'tables':tmp['table_id'].values}
    
        max_depth = max(d['depth'] for d in tables_dict.values())
        tables = [d['tables'] for d in tables_dict.values() if d['depth'] == max_depth]
        tables = list(set(itertools.chain.from_iterable(tables)))
        return tables

    def answer_table(self):
        targets_tables = self.get_tables()
        if len(targets_tables) == 1:
            return targets_tables 
        else:
            tables_score = {}
            text = ''
            for table in targets_tables:
                html = table.split('-')[1]
                if not os.path.exists(f'../data/{self.doc_id}'):
                    os.mkdir(f'../data/{self.doc_id}')
                
                if os.path.exists(f'../data/{self.doc_id}/{table}.pickle'):                   
                    with open(f'../data/{self.doc_id}/{table}.pickle', 'r',) as f:
                        text = f.read()
                        f.close()
                else:
                    with open(f"../test/reports/{self.doc_id}/{html}.html", 'rb') as f:
                        soup = bs(f, 'html.parser')
                        f.close()
                    tags = soup.find_all(['table', 'p', 'h3', 'h4', 'h5'])
                    p_text = ''
                    text = ''
                    tmp_h3 = ''
                    tmp_h4 = ''
                    tmp_h5 = ''
                    for tag in tags:
                        if tag.name == 'h3':
                            tmp_h3 = unicodedata.normalize('NFKC', tag.get_text()).replace('\n', '')
                            tmp_h4 = ''
                            tmp_h5 = ''
                        if tag.name == 'h4':
                            tmp_h4 = unicodedata.normalize('NFKC', tag.get_text()).replace('\n', '')
                            tmp_h5 = ''
                        if tag.name == 'h5':
                            tmp_h5 = unicodedata.normalize('NFKC', tag.get_text()).replace('\n', '')
                        if tag.name == 'table':
                            if tag.get('table-id') == table:
                                tab_text = unicodedata.normalize("NFKC", tag.get_text()).replace("\n", "")
                                text = f'{tmp_h3}\n{tmp_h4}\n{tmp_h5}\n{tab_text}\n{p_text}'
                                break
                            else:
                                p_text = ''
                        else:
                            if tag.parent.name != 'td':
                                p_text = unicodedata.normalize('NFKC', tag.get_text()).replace('\n', '').replace(' ', '')
                    with open(f'../data/{self.doc_id}/{table}.pickle', 'w', encoding='utf-8') as f2:
                        f2.write(text)
                        f2.close()
                tables_score[table] = sum([(word in text) for word in self.words])
            return [kv[0] for kv in tables_score.items() if kv[1] == max(tables_score.values())]

    def edit_question(self, ):
        year = re.search('20(\d+)', self.question).group()
        if year == '2024':
            question=re.sub(f'{year}年', '最新', self.question)
        elif year == '2023':
            question=re.sub(f'{year}年', '前年', self.question)
        elif year == '2022':
            question=re.sub(f'{year}年', '前々年', self.question)
        elif year == '2021':
            question=re.sub(f'{year}年', '前前々年', self.question)
        elif year == '2020':
            question=re.sub(f'{year}年', '前前前々年', self.question)

        if re.search('「(.*)」', self.question):
            words = re.search('「(.*)」', self.question).group()[1:-1].split('、')[::-1]
            question = re.sub('「(.*)」', 'の'.join(words), question)

        if re.search('[a-zA-Z]+', self.question): 
            prompt =f"次単語を日本語に変換して答えのみを出力してください。人名も漢字をあててください。\n{re.search('[a-zA-Z]+', self.question).group()}"
            messages=[
                {"role": "system", "content": "あなたは翻訳家です。"},
                {"role": "user", "content": prompt},
            ]

            response = plamo.chat.completions.create(
                model="plamo-beta",
                #model=engine, 
                messages = messages,
            )
            question = re.sub(re.search('[a-zA-Z]+', self.question).group(), response.choices[0].message.content, question)
        return question
    
    #ChatGPTに質問を投げて、表から回答を抽出する
    def answer(self, ):
        tables = self.answer_table()
        if len(tables) == 1:
            return tables[0]
        elif len(tables) > 5:
            return tables[0]
        else:
            table_text = ''
            for i, table in enumerate(tables):
                if os.path.exists(f'../data/{self.doc_id}/{table}.pickle'):                   
                    with open(f'../data/{self.doc_id}/{table}.pickle', 'r',) as f:
                        table_text +=f"""##Table{i}\n{f.read()}\n"""
                        f.close()
        
            prompt =f"質問に回答が記載されている表の番号を教えてください。特に明記されていない場合は提出会社の決算を答えてください。\n##question\n{self.edit_question()}\n{table_text}\n##出力結果\nTable_num:<Table_num>"
            
            messages=[
                {"role": "system", "content": "あなたは情報抽出タスクを行うタスクを任されてました。"},
                {"role": "user", "content": prompt},
            ]

            response = gpt.chat.completions.create(
                #model="plamo-beta",
                model=engine,
                messages = messages,
            )
            try:
                return tables[int(re.search('\d', response.choices[0].message.content).group())]
            except:
                return table[0]

In [52]:
df_train = data.query('flag in ["train",]')
question, words, doc_id, table_id = data.query('question_number =="question_tr_valid2"')[['question',"words","doc_id", 'table_id']].values[0]
words = json.loads(words.replace("'", '"'))
print(question)
print(table_id)
print(search_table(words, doc_id, df_train, question).answer())

日本光電工業株式会社の2020年における「当期純利益又は当期純損失（△）、経営指標等」は？
S100TUGR-0101010-tab3
2
Table_num:1


In [ ]:
from tqdm import tqdm
import time
df_train = data.query('flag in ["train", "valid"]')
answer_sheet = {}
for index, row in tqdm(data.query('flag =="test"')[344:].iterrows()):
    print(row['question_number'])
    words, doc_id, table_id = row[["words","doc_id", 'table_id']]
    words = json.loads(words.replace("'", '"'))
    answer = search_table(words, doc_id, df_train, question).answer()
    answer_sheet[row['question_number']] = answer
    time.sleep(1)

0it [00:00, ?it/s]

question_tr_test345


1it [00:03,  3.50s/it]

question_tr_test346


2it [00:07,  3.62s/it]

...

question_tr_test390


46it [05:07,  5.10s/it]

question_tr_test391


47it [05:12,  4.94s/it]

question_tr_test392


In [56]:
total = 0
correct = 0
gold_sd = dict(data.query('flag == "valid"')[['question_number', 'table_id']].values)
for key in answer_sheet.keys():
    total += 1
    if gold_sd[key] in answer_sheet[key]:
        correct += 1
    else:
        print(f'{key}, TRUE:{gold_sd[key]}, PRED:{answer_sheet[key]}')
        pass
        
print(correct/total)

question_tr_valid7, TRUE:S100TL6G-0105100-tab162, PRED:S100TL6G-0105050-tab129
question_tr_valid8, TRUE:S100T4ZN-0104010-tab92, PRED:S100T4ZN-0104010-tab91
question_tr_valid14, TRUE:S100TLL3-0105100-tab81, PRED:S100TLL3-0105010-tab70
question_tr_valid17, TRUE:S100TVWH-0105010-tab137, PRED:S100TVWH-0105010-tab118
question_tr_valid19, TRUE:S100TTWS-0105010-tab163, PRED:S100TTWS-0105010-tab164
question_tr_valid20, TRUE:S100TPPL-0104010-tab112, PRED:S100TPPL-0104010-tab116
question_tr_valid44, TRUE:S100TPPL-0105330-tab266, PRED:S100TPPL-0105330-tab267
question_tr_valid47, TRUE:S100TNH4-0105040-tab83, PRED:S100TNH4-0105040-tab85
0.84


In [ ]:
json.dump(answer_sheet,open('../test/answersheet2.json', 'w') ,indent=0)